In [20]:
import torch
import gzip
import numpy as np

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    return text_tokens

def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            new_samples.append([original_sample[0], similar, original_sample[2][0]])# change this to include all negative 
                                                                                   # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]]={'title':token[1],'question':token[2]}
    return lookup
        
    

#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, neg_id]
    titles = []
    for sample_id in sample_ids:
        titles.append([lookup[str(identity)]['title'] for identity in sample_id])
    return titles
    

In [22]:
#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train = convert_to_samples(train_random_filename)
dev = convert_to_samples(dev_filename)
test = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)

train = convert_sampleids_to_titles(train, lookup)
print train
train[:2]

KeyError: '2'

In [18]:
word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()

def extract_features(word):
    word=word.encode('utf-8')
    return word_to_vec.get(word,[0.0 for i in range(200)])

In [9]:
extract_features('laptop')

array([  6.81100000e-03,  -1.91690000e-02,   3.72350000e-02,
         1.88340000e-02,   7.88700000e-03,  -6.64640000e-02,
         8.59540000e-02,  -8.24730000e-02,   1.83440000e-02,
        -8.74610000e-02,  -1.11447000e-01,  -3.71180000e-02,
        -8.30300000e-03,   1.40190000e-02,   8.45250000e-02,
         2.03400000e-03,  -2.82450000e-02,  -1.62900000e-02,
         7.20170000e-02,   6.04480000e-02,  -1.39264000e-01,
        -1.63540000e-02,   1.74352000e-01,   3.68500000e-03,
        -9.88580000e-02,  -4.12110000e-02,  -4.93310000e-02,
        -2.85100000e-02,  -9.08550000e-02,  -2.40180000e-02,
        -9.21600000e-03,  -3.20450000e-02,  -3.30730000e-02,
        -2.61440000e-02,   1.22893000e-01,  -3.29470000e-02,
        -9.37550000e-02,   1.87461000e-01,   1.35400000e-03,
         7.04130000e-02,   3.89100000e-03,  -2.50700000e-02,
        -1.38250000e-02,   5.29970000e-02,  -9.65200000e-02,
         5.73710000e-02,  -1.60480000e-02,   3.27330000e-02,
         5.89300000e-02,

In [26]:

print sample_ids

NameError: name 'sample_ids' is not defined